In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from random import gauss, seed
from scipy import stats
import seaborn as sns
from mpl_toolkits import mplot3d
import sklearn.metrics as metrics
import statsmodels.api as sm
from sklearn.model_selection import train_test_split

%matplotlib inline

from sklearn.utils import resample
from sklearn.datasets import load_breast_cancer, load_iris, make_classification
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import confusion_matrix, plot_confusion_matrix,\
    precision_score, recall_score, accuracy_score, f1_score, log_loss,\
    roc_curve, roc_auc_score, classification_report


In [2]:
# reading CSVs
vehicle_df = pd.read_csv('../data/localdata/Traffic_Crashes_Vehicles.csv')
people_df = pd.read_csv('../data/localdata/Traffic_Crashes_People.csv')
crash_df = pd.read_csv('../data/localdata/Traffic_Crashes_Crashes.csv')

/Users/robertdaly/anaconda3/envs/learn-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (21,40,41,42,44,48,49,50,53,55,58,59,61,71) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/robertdaly/anaconda3/envs/learn-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (20,24,25,26,29) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
people_df.head()

,PERSON_ID,PERSON_TYPE,CRASH_RECORD_ID,RD_NO,VEHICLE_ID,CRASH_DATE,SEAT_NO,CITY,STATE,ZIPCODE,...,EMS_RUN_NO,DRIVER_ACTION,DRIVER_VISION,PHYSICAL_CONDITION,PEDPEDAL_ACTION,PEDPEDAL_VISIBILITY,PEDPEDAL_LOCATION,BAC_RESULT,BAC_RESULT VALUE,CELL_PHONE_USE
0,O749947,DRIVER,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,JC451435,834816.0,09/28/2019 03:30:00 AM,NaN,CHICAGO,IL,60651,...,NaN,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
1,O871921,DRIVER,af84fb5c8d996fcd3aefd36593c3a02e6e7509eeb27568...,JD208731,827212.0,04/13/2020 10:50:00 PM,NaN,CHICAGO,IL,60620,...,NaN,NONE,NOT OBSCURED,NORMAL,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
2,O10018,DRIVER,71162af7bf22799b776547132ebf134b5b438dcf3dac6b...,HY484534,9579.0,11/01/2015 05:00:00 AM,NaN,NaN,NaN,NaN,...,NaN,IMPROPER BACKING,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
3,O10038,DRIVER,c21c476e2ccc41af550b5d858d22aaac4ffc88745a1700...,HY484750,9598.0,11/01/2015 08:00:00 AM,NaN,NaN,NaN,NaN,...,NaN,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
4,O10039,DRIVER,eb390a4c8e114c69488f5fb8a097fe629f5a92fd528cf4...,HY484778,9600.0,11/01/2015 10:15:00 AM,NaN,NaN,NaN,NaN,...,NaN,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN


In [4]:
people_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1653183 entries, 0 to 1653182
Data columns (total 30 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   PERSON_ID              1653183 non-null  object 
 1   PERSON_TYPE            1653183 non-null  object 
 2   CRASH_RECORD_ID        1653183 non-null  object 
 3   RD_NO                  1643794 non-null  object 
 4   VEHICLE_ID             1620516 non-null  float64
 5   CRASH_DATE             1653183 non-null  object 
 6   SEAT_NO                334832 non-null   float64
 7   CITY                   1204832 non-null  object 
 8   STATE                  1222001 non-null  object 
 9   ZIPCODE                1103358 non-null  object 
 10  SEX                    1626486 non-null  object 
 11  AGE                    1171015 non-null  float64
 12  DRIVERS_LICENSE_STATE  969912 non-null   object 
 13  DRIVERS_LICENSE_CLASS  815538 non-null   object 
 14  SAFETY_EQUIPMENT  

In [5]:
vehicles_df.info()

NameError: name 'vehicles_df' is not defined

In [ ]:
crashes_df.info()

In [ ]:
crashes_df['PRIM_CONTRIBUTORY_CAUSE'].value_counts()

In [ ]:
crashes_df['SEC_CONTRIBUTORY_CAUSE'].value_counts()

In [ ]:
crashes_df['DAMAGE'].value_counts()

In [ ]:
crashes_df['INJURIES_FATAL'].value_counts()

In [ ]:
crashes_df['NOT_RIGHT_OF_WAY_I'].value_counts()

In [ ]:
crashes_df['CRASH_DAY_OF_WEEK'].value_counts()

In [ ]:
crashes_df['INTERSECTION_RELATED_I'].value_counts()

In [ ]:
crashes_df['TRAFFICWAY_TYPE'].value_counts()

In [ ]:
crashes_df['LANE_CNT'].value_counts()

In [ ]:
crashes_df['ALIGNMENT'].value_counts()

In [ ]:
crashes_df['ROADWAY_SURFACE_COND'].value_counts()

In [ ]:
crashes_df['ROAD_DEFECT'].value_counts()

In [ ]:
crashes_df['CRASH_TYPE'].value_counts()

In [ ]:
crashes_df['CRASH_MONTH'].value_counts()

In [ ]:
crashes_df.drop(columns=['WORK_ZONE_I', 'WORK_ZONE_TYPE', 'DOORING_I', 'WORKERS_PRESENT_I',
                                          'PHOTOS_TAKEN_I', 'STATEMENTS_TAKEN_I', 'RD_NO'], axis=1, inplace=True)


In [ ]:
crashes_df.dropna(subset = ['INJURIES_TOTAL', 'LATITUDE', 'MOST_SEVERE_INJURY', 'STREET_DIRECTION', 'BEAT_OF_OCCURRENCE'] , axis=0, inplace=True)

In [ ]:
crashes_df = crashes_df[(crashes_df['LANE_CNT'] <= 10)]

In [ ]:
crashes_df['NUM_UNITS'].value_counts()

In [ ]:
vehicles_df.info()

In [ ]:
vehicles_df['VEHICLE_YEAR'].value_counts()

In [ ]:
vehicles_df['MANEUVER'].value_counts()

In [ ]:
crashes_df['WEATHER_CONDITION'].value_counts()

In [ ]:
crashes_df['LIGHTING_CONDITION'].value_counts()

In [ ]:
vehicles_df['AREA_00_I'].value_counts()

In [ ]:
vehicles_df.columns = vehicles_df.columns.str.title()
vehicles_df.columns = vehicles_df.columns.str.replace('_', ' ')
vehicles_df.info()

In [ ]:
area_columns = vehicles_df.columns[vehicles_df.columns.str.contains('Area', case=False)]
area_df = vehicles_df[area_columns]


In [ ]:
area_df.describe()

In [ ]:
vehicles_df['Exceed Speed Limit I'].value_counts()

In [ ]:
vehicles_df['Travel Direction'].value_counts()

In [ ]:
vehicles_df['Carrier Name'].value_counts()

In [ ]:
vehicles_df['Hazmat Report I'].value_counts()

In [ ]:
vehicles_df['Hazmat Vio Cause Crash I'].value_counts()

In [ ]:
crashes_df['INTERSECTION_RELATED_I'].value_counts()

In [ ]:
crashes_df['INJURIES_TOTAL'].value_counts()

In [ ]:
vehicles_df['Occupant Cnt'].value_counts()

In [ ]:
people_df['SEAT_NO'].value_counts()

In [ ]:
people_df['AGE'].describe()

In [ ]:
people_df['INJURY_CLASSIFICATION'].value_counts()

In [ ]:
people_df['DRIVER_ACTION'].value_counts()

In [ ]:
people_df['DRIVER_VISION'].value_counts()

In [ ]:
people_df['PHYSICAL_CONDITION'].value_counts()

In [ ]:
people_df['BAC_RESULT'].value_counts()

In [ ]:
people_df['PEDPEDAL_LOCATION'].value_counts()

In [ ]:
people_df['PEDPEDAL_ACTION'].value_counts()

In [ ]:
crashes_df['INJURIES_TOTAL'].value_counts()

In [ ]:
people_df['ZIPCODE'].value_counts()

In [ ]:
people_df['AIRBAG_DEPLOYED'].value_counts()

In [ ]:
people_df['SAFETY_EQUIPMENT'].value_counts()

In [ ]:
people_df['EJECTION'].value_counts()

In [ ]:
people_df['HOSPITAL'].value_counts()

In [ ]:
vehicles_df['Vehicle Use'].value_counts()

In [ ]:
vehicles_df['Vehicle Defect'].value_counts()

In [ ]:
vehicles_df['Make'].value_counts()

In [ ]:
vehicles_df

In [ ]:
area_columns = vehicles_df.columns[vehicles_df.columns.str.contains('Area', case=False)]

In [ ]:
vehicles_df[area_columns] = vehicles_df[area_columns].fillna(0).mask(vehicles_df[area_columns] == 'Y', 'Y')

In [ ]:
vehicles_df[area_columns]

In [ ]:
vehicles_df[area_columns] = vehicles_df[area_columns].replace(to_replace="N", value = 0)
vehicles_df[area_columns] = vehicles_df[area_columns].replace(to_replace="Y", value = 1)

In [ ]:
b

In [ ]:
vehicles_df[area_columns].loc[vehicles_df[area_columns].sum(axis=1) > 1]

In [ ]:
vehicles_df['Num Areas'] = vehicles_df[area_columns].sum(axis=1)

In [ ]:
vehicles_df['Num Areas'].value_counts()

In [ ]:
# Injury Score column is a linear combination of the factors that make up Injuries Total simply 
# multiplied by a constant depending on their severity
crashes_df['Injury Score'] = crashes_df['Injuries Fatal']*7 +
                            crashes_df['Injuries Incapacitating']*3 + 
                            crashes_df['Injuries Non Incapacitating']*2 + 
                            crashes_df['Injuries Reported Not Evident']

In [ ]:
# Injury Score

crashes_df[['Injuries Total', 'Injuries Fatal', 'Injuries Incapacitating', 
            'Injuries Non Incapacitating', 'Injuries Reported Not Evident']][crashes_df['Injuries Total']!=0][:50]

crashes_df['total injured'] = crashes_df['Injuries Fatal'] + crashes_df['Injuries Incapacitating'] + crashes_df['Injuries Non Incapacitating'] + crashes_df['Injuries Reported Not Evident']

# Checking if added up columns in 'total injured' make up 'Injuries Total'
(crashes_df['total injured'] == crashes_df['Injuries Total']).value_counts()

# Injury Score column is a linear combination of the factors that make up Injuries Total simply multiplied by a constant depending on their severity
crashes_df['Injury Score'] = crashes_df['Injuries Fatal']*7 + crashes_df['Injuries Incapacitating']*3 + crashes_df['Injuries Non Incapacitating']*2 + crashes_df['Injuries Reported Not Evident']

# Accounting Material Damage of Car and Surrounding Areas (Crash Score)

crashes_df['Damage'].value_counts()
mapping = {'OVER $1,500': 3, '$501 - $1,500': 2, "$500 OR LESS": 1}
crashes_df['Damage_ODE'] = crashes_df['Damage'].map(mapping)
crashes_df['Injury Score'] = crashes_df['Injury Score'] + crashes_df['Damage_ODE']*3
crashes_df['Crash Score'] = crashes_df['Injury Score']